## Initial Modeling Attempt: Small Business Loans with Random Forest

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

In [2]:
df = pd.read_csv("./Data/Processed/sba_cleaned.csv")

## Preparing the Data

To start, we load in the cleaned data from our initial data wrangling.

In [3]:
## Recall the feature names from our data.
df.columns

Index(['ID', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS',
       'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob',
       'RetainedJob', 'FranchiseCode', 'UrbanRural', 'LowDoc', 'ChgOffDate',
       'DisbursementDate', 'DisbursementGross', 'BalanceGross', 'MIS_Status',
       'ChgOffPrinGr', 'GrAppv', 'SBA_Appv'],
      dtype='object')

In [4]:
##It appears that I've accidentally added an extra column called `Unnamed: 0`! Let's remove it.
df = df.drop(df.columns[0], axis = 1)

We choose the features to use in our model. Some features, such as `ChgOffDate` and clearly relate to the eventual fate of the loan, so are not appropriate to use in our model. Similarly, we drop `DisbursementDate`, `DisbursementGross`, `BalanceGross` and `ChgOffPrinGr`. It is not obvious whether CreateJob and RetainedJob refer to projections from the loan application, or later follow-up. We leave them in for now. To simplify the initial model, we drop `ApprovalDate` keep only `ApprovalFY`.

We also remove the `ID`, `City`, `Zip`, `Bank`, `BankState` and `FranchiseCode.` These categorical variables have a large number of values, which would create memory issues with one-hot encoding.

We subset on the remaining columns, and drop rows with missing values.

In [5]:
## Create list of features to use for our model
features = ['State', 'NAICS', 'Appro valFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'UrbanRural','LowDoc', 'MIS_Status', 'GrAppv', 'SBA_Appv']

In [6]:
## Subset on relevant columns
df_pred = df[features]

KeyError: "['Appro valFY'] not in index"

In [ ]:
df_pred.isna().sum()

In [ ]:
df_pred.shape

In [ ]:
import matplotlib.pyplot as plt
x = df_pred.RetainedJob.value_counts()
x

In [ ]:
## Drop rows with missing values
df_pred = df_pred.dropna(axis = 0)

Since csv format does not keep track of data stypes, we must examine those and reset as needed.

In [ ]:
## Examine the data types.
df_pred.dtypes

In [ ]:
## Columns to be converted to categories
cat_cols = ['State', 'NAICS', 'NewExist', 'UrbanRural', 'LowDoc', 'MIS_Status']
df_pred[cat_cols] = df_pred[cat_cols].apply(lambda x: x.astype('category'))

In [ ]:
## Save approval year as an integer
df_pred['ApprovalFY'] = df_pred['ApprovalFY'].apply(lambda x: int(x[:4]))

In [ ]:
df_pred.dtypes

There are are a large number of NAICS codes, which may create memory issues with one-hot encoding. However, the first two digits of the NAICS code keeps track of the overeall type of industry. We simplify by replacing NAICS code with its first two digits.

In [ ]:
## First two digits of each industry code
df_pred['Industry'] = df_pred['NAICS'].apply(lambda x: str(x)[:2])

In [ ]:
df_pred['Industry'] = df_pred['Industry'].astype('category')

In [ ]:
df_pred = df_pred.drop('NAICS', axis = 1)

We use dummy variables to encode categorical variables as numeric. 

In [ ]:
## Encode categorical features
df_features = pd.get_dummies(df_pred)

In [ ]:
## Inspect our new list of features
df_features.columns
len(df_features.columns)

In [ ]:
## Dummy variabes encoding creates two columns corresponding to MIS Status.
## Drop the MIS_Status_PIF column
df_features = df_features.drop(df_features.columns[-25], axis = 1)

In [ ]:
## Array of predictors
X = df_features.drop('MIS_Status_CHGOFF', axis = 1).values
labels = df_features.drop('MIS_Status_CHGOFF', axis = 1).columns

In [ ]:
## Column of labels
y = df_features['MIS_Status_CHGOFF'].values

## Training the Model

We train a random forrest model, using 20% of our data as a training set. We scale the features using standard scalar.

Question: should the scalar be applied to columns representing categorical data?

In [ ]:
## Split data in to test and train
## Switch to 70% training, 30 test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
## Fit a Random Forrest model, making sure to scale the data first
RF_pipe = make_pipeline(
    StandardScaler(),
    RandomForestClassifier(random_state = 0)
)

In [ ]:
## Fit the model
RF_pipe.fit(X_train, y_train)

In [ ]:
## Make predictions
y_te_pred = RF_pipe.predict(X_test)

In [ ]:
## Acurracy score on the test set is nearly 93%. Not bad for a first try!
accuracy_score(y_test, y_te_pred)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_te_pred)

In [ ]:
RF_pipe.named_steps

### Feature Importances

What features are contributing most to the model? What does this tell us?

In [ ]:
importances = RF_pipe['randomforestclassifier'].feature_importances_

## Create a data frame listing feature names and importances
features_ranked = pd.DataFrame(zip(labels, importances), columns = ['feature name', 'importance'])

In [ ]:
## Print a table showing the tope 20 features
features_ranked = features_ranked.sort_values('importance', ascending = False)
features_ranked.head(20)

In [ ]:
import seaborn as sns
plt.figure(figsize=(10,8))
top_features = features_ranked[:20]
sns.barplot(x=top_features['importance'], y=top_features['feature name'], color = 'lightblue')
#Add chart labels
plt.title('Random Forrest Feature Importance')
plt.xlabel('Importance')
plt.ylabel('Name')

We note that fiscal year is one of the most important features! This is concerning--how will a model that depends on year-by-year trends be able to predict future data? We need to account for variation between years somehow.

## Correlations among top features

Let's see which--if any--of the top features correlate with each other. (Question: Should I remove redundant "dummy variables"?

In [ ]:
all_cols = list(top_features['feature name'])
all_cols.append('MIS_Status_CHGOFF')
top_cols = df_features[all_cols]

In [ ]:
## Plot a heatmap of top twenty features, together with loan status. There are only a few very strong correlations here.
plt.figure(figsize = (8, 8))
ax = sns.heatmap(top_cols.corr(), xticklabels=True, yticklabels=True, square = True)

In [ ]:
by_year = df_features.groupby('ApprovalFY')['MIS_Status_CHGOFF'].mean()

In [ ]:
## Plot default rates by year. We see that there is a large spike for loans approved in mid-80's, 
## and another large one for laons approved in the late 2000's.
ax = by_year.plot(ylabel = 'Proportion of loans defaulted')

In [ ]:
year_2007 = df_features.loc[df_features['ApprovalFY'] == 2007]

year_2007.head(10)